In [2]:
import logging
import numpy as np
from pricing_market_logic_multiproduct import (
    get_quantities,
    get_profits,
    get_monopoly_prices,
    get_nash_prices,
)

class CalvanoDemandEnvironment:
    def __init__(self, name: str, description: str, nbr_agents: int, env_params: dict, logger: logging.Logger = None):
        self.name = name
        self.description = description
        self.nbr_agents = nbr_agents
        self.logger = logger or logging.getLogger("experiment_logger")

        self.a_0 = env_params.get("a_0", 0.0)
        self.a = env_params.get("a", np.ones(nbr_agents))
        self.mu = env_params.get("mu", 0.25)
        self.alpha = env_params.get("alpha", np.ones(nbr_agents))
        self.beta = env_params.get("beta", 100)
        self.sigma = env_params.get("sigma", 0.0)
        self.c = env_params.get("c", np.ones(nbr_agents))
        self.group_idxs = env_params.get("group_idxs", [])

        self.monopoly_prices = None
        self.nash_prices = None
        self._compute_benchmarks()


    def compute_quantities_and_profits(self, prices: dict[str, float]) -> tuple[dict[str, float], dict[str, float]]:
        try:
            # Ensure ordering by agent index
            agent_names = list(prices.keys())
            price_values = [prices[name] for name in agent_names]

            quantities = get_quantities(
                p=tuple(price_values), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            profits = get_profits(
                p=tuple(price_values), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, c=self.c, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            # Return results mapped back to agent names
            quantities_dict = {name: q for name, q in zip(agent_names, quantities)}
            profits_dict = {name: pi for name, pi in zip(agent_names, profits)}
            return quantities_dict, profits_dict
        except Exception as e:
            self.logger.error(f"Error computing quantities and profits: {e}")
            raise e
    
    def _compute_benchmarks(self):
        try:
            # Monopoly solution
            self.monopoly_prices = get_monopoly_prices(
                a0=self.a_0, a=self.a, mu=self.mu, alpha=self.alpha,
                c=self.c, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            self.monopoly_quantities = get_quantities(
                p=tuple(self.monopoly_prices), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            self.monopoly_profits = get_profits(
                p=tuple(self.monopoly_prices), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, c=self.c, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            # Nash solution
            self.nash_prices = get_nash_prices(
                a0=self.a_0, a=self.a, mu=self.mu, alpha=self.alpha, c=self.c,
                multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            self.nash_quantities = get_quantities(
                p=tuple(self.nash_prices), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )

            self.nash_profits = get_profits(
                p=tuple(self.nash_prices), a0=self.a_0, a=self.a, mu=self.mu,
                alpha=self.alpha, c=self.c, multiplier=self.beta, sigma=self.sigma, group_idxs=self.group_idxs,
            )
        except Exception as e:
            self.logger.error(f"Error computing benchmarks when initializing environment: {e}")
            raise e
    
    def get_environment_params(self):
        return {
            "a_0": self.a_0,
            "a": self.a.tolist(),
            "mu": self.mu,
            "alpha": self.alpha.tolist(),
            "beta": self.beta,
            "sigma": self.sigma,
            "c": self.c.tolist(),
            "group_idxs": self.group_idxs,
            "monopoly_prices": self.monopoly_prices if self.monopoly_prices is not None else None,
            "monopoly_quantities": self.monopoly_quantities if self.monopoly_quantities is not None else None,
            "monopoly_profits": self.monopoly_profits if self.monopoly_profits is not None else None,
            "nash_prices": self.nash_prices if self.nash_prices is not None else None,
            "nash_quantities": self.nash_quantities if self.nash_quantities is not None else None,
            "nash_profits": self.nash_profits if self.nash_profits is not None else None,
        }


In [12]:
alpha = 1
env_params = {
    "a_0": 0.0,
    "a": np.array([2.0, 2.0]),
    "mu": 0.25,
    "alpha": np.array([alpha, alpha]),
    "beta": 100,
    "sigma": 0.25,
    "c": np.array([1.0, 1.0]), #this is ok as they do not affect
    "group_idxs": (1,2),
}

env = CalvanoDemandEnvironment(name="", description="", nbr_agents=2, env_params=env_params)
env.get_environment_params()

{'a_0': 0.0,
 'a': [2.0, 2.0],
 'mu': 0.25,
 'alpha': [1, 1],
 'beta': 100,
 'sigma': 0.25,
 'c': [1.0, 1.0],
 'group_idxs': (1, 2),
 'monopoly_prices': [1.8859367839279824, 1.8859367841816808],
 'monopoly_quantities': [35.89064366090311, 35.89064362448151],
 'monopoly_profits': [np.float64(31.796841418045727),
  np.float64(31.796841394883895)],
 'nash_prices': [1.467837558344815, 1.4678375571705864],
 'nash_quantities': [46.56244983052504, 46.56245004922491],
 'nash_profits': [np.float64(21.783662839265784),
  np.float64(21.783662886906832)]}

In [13]:
alpha = 1
env_params = {
    "a_0": 0.0,
    "a": np.array([2.0, 2.0]),
    "mu": 0.25,
    "alpha": np.array([alpha, alpha]),
    "beta": 100,
    "sigma": 0.25,
    "c": np.array([1.0, 1.0]), #this is ok as they do not affect
    "group_idxs": (1,1),
}

env = CalvanoDemandEnvironment(name="", description="", nbr_agents=2, env_params=env_params)
env.get_environment_params()

{'a_0': 0.0,
 'a': [2.0, 2.0],
 'mu': 0.25,
 'alpha': [1, 1],
 'beta': 100,
 'sigma': 0.25,
 'c': [1.0, 1.0],
 'group_idxs': (1, 1),
 'monopoly_prices': [1.8550587622636872, 1.8550587622636872],
 'monopoly_quantities': [35.381112567295425, 35.381112567295425],
 'monopoly_profits': [np.float64(30.25293031930381),
  np.float64(30.25293031930381)],
 'nash_prices': [1.3603933375410198, 1.3603933386362583],
 'nash_quantities': [47.29812718551157, 47.29812690923032],
 'nash_profits': [np.float64(17.045929915826154),
  np.float64(17.045929868058966)]}

In [22]:
0.22* 1/np.exp(1)

np.float64(0.08093347705771732)